In [21]:
!brew install deno

To reinstall 2.4.5, run:
  brew reinstall deno


In [22]:
!deno install

In [23]:
%pip install dspy langchain langchain-community yfinance

Note: you may need to restart the kernel to use updated packages.


In [24]:
# DSPy configuration for Financial Analyst agent
import os
import dspy
import yfinance as yf

# Optionally load environment from .env if available
try:
    from dotenv import load_dotenv  # type: ignore
    load_dotenv()
except Exception:
    pass

# Configure model provider (OpenAI-only, consistent with other agents)
lm = dspy.LM("openai/gpt-5-mini", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000)

dspy.configure(lm=lm)

print("DSPy configured for Financial Analyst agent.")


DSPy configured for Financial Analyst agent.


In [25]:
# Configure Deno for DSPy ProgramOfThought (use workspace deno.json)
import os, shutil, pathlib

deno_cfg = str((pathlib.Path.cwd() / "deno.json").resolve())
os.environ["DENO_CONFIG"] = deno_cfg

if shutil.which("deno") is None:
    for p in ["/opt/homebrew/bin", "/usr/local/bin"]:
        if p not in os.environ.get("PATH", ""):
            os.environ["PATH"] = os.environ.get("PATH", "") + f":{p}"

print("DENO_CONFIG:", os.environ.get("DENO_CONFIG"))
print("deno in PATH:", shutil.which("deno"))


DENO_CONFIG: /Users/hammermt/Codes/ai-agents-udemy-course/dspy/financial_analyst_agent/deno.json
deno in PATH: /opt/homebrew/bin/deno


In [26]:
# Yahoo Finance tools (simple, deterministic wrappers)
from typing import List, Optional, Literal, Dict, Any


def yf_get_price(ticker: str) -> str:
    """Return latest price info for a ticker as a compact string.
    Designed for tool use within ReAct.
    """
    if not ticker or not isinstance(ticker, str):
        return "Invalid ticker"
    try:
        t = yf.Ticker(ticker)
        info = t.fast_info if hasattr(t, "fast_info") else {}
        last = getattr(info, "last_price", None) if hasattr(info, "last_price") else info.get("last_price") if isinstance(info, dict) else None
        currency = getattr(info, "currency", None) if hasattr(info, "currency") else info.get("currency") if isinstance(info, dict) else None
        if last is None:
            data = t.history(period="1d")
            if data is None or data.empty:
                return f"No price data for {ticker}."
            last = float(data["Close"].iloc[-1])
        return f"{ticker} price: {last} {currency or ''}".strip()
    except Exception as e:
        return f"Error fetching price for {ticker}: {e}"


def yf_get_history(ticker: str, period: str = "5d", interval: str = "1d") -> str:
    """Return recent historical OHLCV rows in a compact, newline-separated string.
    Keep output concise for model consumption in ReAct.
    """
    if not ticker:
        return "Invalid ticker"
    try:
        # Use Ticker().history to avoid MultiIndex columns from download()
        t = yf.Ticker(ticker)
        data = t.history(period=period, interval=interval, auto_adjust=False)
        if data is None or data.empty:
            return f"No history for {ticker}."
        # Keep last up to 10 rows for brevity
        tail = data.tail(10)
        lines = []
        for idx, row in tail.iterrows():
            try:
                o = float(row["Open"]) if row.get("Open") is not None else None
                h = float(row["High"]) if row.get("High") is not None else None
                l = float(row["Low"]) if row.get("Low") is not None else None
                c = float(row["Close"]) if row.get("Close") is not None else None
                v_raw = row.get("Volume")
                v = int(v_raw) if v_raw is not None and not (hasattr(v_raw, "isna") and v_raw.isna()) else 0
                o_str = f"{o:.2f}" if o is not None else ""
                h_str = f"{h:.2f}" if h is not None else ""
                l_str = f"{l:.2f}" if l is not None else ""
                c_str = f"{c:.2f}" if c is not None else ""
                v_str = str(v)
            except Exception:
                o_str = str(row.get("Open", ""))
                h_str = str(row.get("High", ""))
                l_str = str(row.get("Low", ""))
                c_str = str(row.get("Close", ""))
                v_str = str(row.get("Volume", ""))
            lines.append(f"{idx.date()} O={o_str} H={h_str} L={l_str} C={c_str} V={v_str}")
        return f"History {ticker} ({period}, {interval}):\n" + "\n".join(lines)
    except Exception as e:
        return f"Error fetching history for {ticker}: {e}"


In [27]:
# ProgramOfThought: analysis signature + module
# Reference: DSPy ProgramOfThought tutorial
# https://dspy.ai/tutorials/program_of_thought/?h=programofthought#tutorial-programofthought

class FinanceAnalysis(dspy.Signature):
    """Write Python code to analyze provided financial data/questions and output the final numeric or textual result.

    Keep code minimal and safe; print only the final result.
    """
    question = dspy.InputField()
    context = dspy.InputField()
    answer = dspy.OutputField()


# Create the ProgramOfThought program for finance analysis
finance_pot = dspy.ProgramOfThought(FinanceAnalysis)


def run_finance_analysis(question: str, context: str = "") -> str:
    """Helper wrapper to run ProgramOfThought for finance tasks."""
    try:
        pred = finance_pot(question=question, context=context)
        return getattr(pred, "answer", "")
    except Exception as e:
        return f"PoT error: {e}"


In [28]:
# ReAct signature for Financial Analysis

class FinanceReActSignature(dspy.Signature):
    """
    You are a financial analyst. Use tools to fetch market data and perform deterministic analyses.

    Tools available:
    - yf_get_price(ticker: str) -> str  # latest price summary
    - yf_get_history(ticker: str, period: str = "5d", interval: str = "1d") -> str  # recent OHLCV
    - run_finance_analysis(question: str, context: str = "") -> str  # ProgramOfThought analysis

    Behavior:
    - For data retrieval, call yf_get_price / yf_get_history.
    - For calculations, comparisons, or aggregations, call run_finance_analysis with a concise question and include retrieved context.
    - Finish with a concise, well-structured answer.

    Outputs:
    - action: primary tool used (one of: yf_get_price, yf_get_history, run_finance_analysis, answer_direct)
    - tool_result: the most relevant tool output used
    - answer: final financial analysis answer
    """
    user_message: str = dspy.InputField(description="User's finance request")
    history: dspy.History = dspy.InputField(description="Conversation history")

    reasoning: str = dspy.OutputField(description="Brief plan of tool calls and why")
    action: str = dspy.OutputField(description="Chosen primary action/tool")
    tool_result: str = dspy.OutputField(description="Tool output used")
    answer: str = dspy.OutputField(description="Final answer")


In [29]:
# FinancialAnalystAgent module using ReAct + ProgramOfThought

class FinancialAnalystAgent(dspy.Module):
    def __init__(self, max_iters: int = 5):
        super().__init__()
        self.conversation_history = dspy.History(messages=[])
        self.react = dspy.ReAct(
            FinanceReActSignature,
            tools=[yf_get_price, yf_get_history, run_finance_analysis],
            max_iters=max_iters,
        )

    def forward(self, user_message: str):
        # Append user message to internal history
        self.conversation_history.messages.append({"role": "user", "content": user_message})
        # Run ReAct with internal history
        result = self.react(user_message=user_message, history=self.conversation_history)
        # Capture assistant response back to history
        answer = getattr(result, "answer", "")
        if isinstance(answer, str) and answer.strip():
            self.conversation_history.messages.append({"role": "assistant", "content": answer})
        return result


In [30]:
# Instantiate the Financial Analyst agent
finance_agent = FinancialAnalystAgent(max_iters=5)
print("FinancialAnalystAgent ready.")


FinancialAnalystAgent ready.


In [31]:
# Example: Compare 5-day returns for two tickers using ReAct tools + ProgramOfThought

# Step 1: Retrieve recent history for two tickers (e.g., AAPL and MSFT)
hist_aapl = yf_get_history("AAPL", period="5d", interval="1d")
hist_msft = yf_get_history("MSFT", period="5d", interval="1d")

print(hist_aapl.splitlines()[0])
print(hist_aapl)
print()
print(hist_msft.splitlines()[0])
print(hist_msft)

# Step 2: Ask ProgramOfThought to compute 5-day percent returns and compare
context = f"AAPL history:\n{hist_aapl}\n\nMSFT history:\n{hist_msft}"
question = (
    "Using the provided daily Close prices for AAPL and MSFT, compute each ticker's 5-day return in percent "
    "(from oldest Close to newest Close), round to two decimals, and then state which outperformed."
)

comparison_result = run_finance_analysis(question=question, context=context)
print("\nPoT comparison result:\n", comparison_result)

# Optional: show a ReAct chat example using the agent (one shot)
resp = finance_agent("Compare AAPL and MSFT 5-day performance and provide the numeric returns.")
print({
    "answer": getattr(resp, "answer", ""),
    "action": getattr(resp, "action", ""),
    "tool_result": getattr(resp, "tool_result", "")[:500],
})


History AAPL (5d, 1d):
History AAPL (5d, 1d):
2025-09-11 O=226.88 H=230.45 L=226.65 C=230.03 V=50208600
2025-09-12 O=229.22 H=234.51 L=229.02 C=234.07 V=55824200
2025-09-15 O=237.00 H=238.19 L=235.03 C=236.70 V=42699500
2025-09-16 O=237.18 H=241.22 L=236.32 C=238.15 V=63384900
2025-09-17 O=238.97 H=240.10 L=237.73 C=238.90 V=28177460

History MSFT (5d, 1d):
History MSFT (5d, 1d):
2025-09-11 O=502.25 H=503.17 L=497.88 C=501.01 V=18881600
2025-09-12 O=506.65 H=512.55 L=503.85 C=509.90 V=23624900
2025-09-15 O=508.79 H=515.47 L=507.00 C=515.36 V=17143800
2025-09-16 O=516.88 H=517.23 L=508.60 C=509.04 V=19688100
2025-09-17 O=510.64 H=510.82 L=505.94 C=509.43 V=8456461

PoT comparison result:
 AAPL 5-day return: 3.86%
MSFT 5-day return: 1.68%
Outperformed: AAPL
{'answer': 'Summary (5 trading days: 2025-09-11 → 2025-09-17)\n- AAPL: Start 230.03 → End 238.90 = +8.87 absolute (+3.856%)\n- MSFT: Start 501.01 → End 510.09 = +9.08 absolute (+1.8123%)\n- Comparison: Both positive; AAPL outperformed

In [32]:
resp.trajectory

{'thought_0': "I'll fetch 5-day historical OHLCV for AAPL first so I can calculate the 5-day returns and then fetch MSFT's history.",
 'tool_name_0': 'yf_get_history',
 'tool_args_0': {'ticker': 'AAPL', 'period': '5d', 'interval': '1d'},
 'observation_0': 'History AAPL (5d, 1d):\n2025-09-11 O=226.88 H=230.45 L=226.65 C=230.03 V=50208600\n2025-09-12 O=229.22 H=234.51 L=229.02 C=234.07 V=55824200\n2025-09-15 O=237.00 H=238.19 L=235.03 C=236.70 V=42699500\n2025-09-16 O=237.18 H=241.22 L=236.32 C=238.15 V=63384900\n2025-09-17 O=238.97 H=240.10 L=237.73 C=238.90 V=28271463',
 'thought_1': 'Now fetch MSFT 5-day historical OHLCV so I can compute its 5-day return and then run the analysis to compare numeric returns for AAPL vs MSFT.',
 'tool_name_1': 'yf_get_history',
 'tool_args_1': {'ticker': 'MSFT', 'period': '5d', 'interval': '1d'},
 'observation_1': 'History MSFT (5d, 1d):\n2025-09-11 O=502.25 H=503.17 L=497.88 C=501.01 V=18881600\n2025-09-12 O=506.65 H=512.55 L=503.85 C=509.90 V=23624900

In [33]:
lm.inspect_history(n=1)





[2025-09-17T20:06:22.996951]

System message:

Your input fields are:
1. `user_message` (str): User's finance request
2. `history` (History): Conversation history
3. `trajectory` (str):
Your output fields are:
1. `reasoning` (str): Brief plan of tool calls and why
2. `action` (str): Chosen primary action/tool
3. `tool_result` (str): Tool output used
4. `answer` (str): Final answer
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## user_message ## ]]
{user_message}

[[ ## history ## ]]
{history}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## action ## ]]
{action}

[[ ## tool_result ## ]]
{tool_result}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are a financial analyst. Use tools to fetch market data and perform deterministic analyses.
        
        Tools available:
        - yf_get_price(ticker: str) -> str  # latest price sum